This Query is created by user sohamjcpp

In [2]:
%% sql
```
WITH 
intermediary_data AS (
  SELECT 
    PCOM_CUSTOMER.CUST_CODE AS intermediary_code,
    PCOM_CUSTOMER.CUST_NAME AS intermediary_name,
    PCOM_CUSTOMER.CUST_TYPE AS intermediary_type,
    PCOM_CUSTOMER.CUST_MOBILE AS agent_mobile_no,
    PCOM_CUSTOMER.CUST_EMAIL AS agent_email_id
  FROM 
    Premia.Pcom_customer PCOM_CUSTOMER
  WHERE 
    SUBSTRING(PCOM_CUSTOMER.CUST_CODE, 1, 2) = dim_intermediary_type
),
mapping_data AS (
  SELECT 
    T_RPDB_POL_PARTY.CUST_CODE,
    T_RPDB_POL_PARTY.PARTY_CODE
  FROM 
    Bancs.T_RPDB_POL_PARTY T_RPDB_POL_PARTY
  INNER JOIN 
    intermediary_data ON T_RPDB_POL_PARTY.CUST_CODE = intermediary_data.intermediary_code
),
preprocessed_data AS (
  SELECT 
    intermediary_data.intermediary_code,
    intermediary_data.intermediary_name,
    intermediary_data.intermediary_type,
    intermediary_data.agent_mobile_no,
    intermediary_data.agent_email_id,
    CASE 
      WHEN DATEDIFF(DAY, intermediary_data.aml_acknowledged_timestamp, GETDATE()) >= 365 THEN 'Yes'
      ELSE 'No'
    END AS due_aml_training,
    DATEADD(DAY, -1, GETDATE()) AS reported_date,
    GETDATE() AS load_timestamp,
    agg_policy_master_report_daily.fufiller_code,
    agg_policy_master_report_daily.fulfiller_name,
    agg_policy_master_report_daily.office_code,
    agg_policy_master_report_daily.office_name,
    agg_policy_master_report_daily.zone_name,
    agg_policy_master_report_daily.mkt_zone_name
  FROM 
    intermediary_data
  LEFT JOIN 
    vo_login ON intermediary_data.intermediary_code = vo_login.intermediary_code
  LEFT JOIN 
    agg_policy_master_report_daily ON intermediary_data.intermediary_code = agg_policy_master_report_daily.intermediary_code
),
final_data AS (
  SELECT 
    preprocessed_data.intermediary_code,
    preprocessed_data.intermediary_name,
    preprocessed_data.intermediary_type,
    preprocessed_data.agent_mobile_no,
    preprocessed_data.agent_email_id,
    preprocessed_data.due_aml_training,
    preprocessed_data.reported_date,
    preprocessed_data.load_timestamp,
    preprocessed_data.fufiller_code,
    preprocessed_data.fulfiller_name,
    preprocessed_data.office_code,
    preprocessed_data.office_name,
    preprocessed_data.zone_name,
    preprocessed_data.mkt_zone_name
  FROM 
    preprocessed_data
  LEFT JOIN 
    mapping_data ON preprocessed_data.intermediary_code = mapping_data.CUST_CODE
)
SELECT 
  final_data.intermediary_code,
  Premia.Pcom_customer.CUST_NAME AS intermediary_name,
  final_data.intermediary_type,
  final_data.agent_mobile_no,
  final_data.agent_email_id,
  final_data.due_aml_training,
  final_data.reported_date,
  final_data.load_timestamp,
  final_data.fufiller_code,
  final_data.fulfiller_name,
  final_data.office_code,
  final_data.office_name,
  final_data.zone_name,
  final_data.mkt_zone_name
FROM 
  final_data
LEFT JOIN 
  Premia.Pcom_customer ON final_data.intermediary_code = Premia.Pcom_customer.CUST_CODE
```